In [1]:
# Import des librairies
import numpy as np
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as mpatches
import seaborn as sns

import nltk
import pickle
import time
import logging
logging.disable(logging.WARNING) # disable WARNING, INFO and DEBUG logging everywhere
import itertools
from fuzzywuzzy import fuzz

C:\Users\DELL\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import re
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords

In [7]:
# Liste de questions-réponses
data = [
    { "question": "Est-ce que je peux inviter un médecin à dîner ?", "reponse": "Oui, mais seulement sous certaines conditions. L'invitation doit être raisonnable, proportionnée et liée à un objectif professionnel clair, comme une présentation scientifique ou une discussion sur un produit. Elle doit également respecter les règles de transparence (enregistrement et déclaration des avantages accordés). De plus, elle ne doit pas être perçue comme une tentative d’influencer les décisions médicales."},
    { "question": "Puis-je offrir un cadeau à un médecin ou à un professionnel de santé ?", "reponse": "Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques."},
    { "question": "Est-il acceptable de financer la participation d’un médecin à un congrès ?", "reponse": "Oui, cela est possible, mais cela doit être clairement lié à une activité scientifique ou professionnelle. Les frais doivent être justifiés, la participation doit être documentée, et le médecin ne doit pas être surchargé par des financements en dehors de ce cadre. Toute contribution doit être publiée selon les exigences de la loi Sapin 2."},
     { "question": "Peut-on offrir une rémunération à un médecin pour une consultation ou une expertise ?", "reponse": "Oui, un paiement pour une expertise scientifique ou médicale est autorisé si cela est fait dans un cadre professionnel et selon des conditions de transparence. Le montant doit être raisonnable et justifiable, et il doit correspondre au temps effectivement passé pour l'exercice de cette activité."},
     { "question": "Est-il permis de prendre en charge les frais de transport d’un médecin pour un événement professionnel ?", "reponse": "Oui, cela est possible si les frais sont directement liés à l’événement professionnel et sont raisonnables. Les frais doivent être documentés, et l'invitation doit respecter les principes de transparence pour éviter tout conflit d’intérêt."},
     { "question": "Puis-je organiser une soirée de gala ou un événement social pour des médecins ?", "reponse": "L’organisation d’un événement social est possible, mais il doit avoir une finalité professionnelle et ne pas être excessive. Les coûts doivent être raisonnables et transparents, et l’événement ne doit pas compromettre l’indépendance du professionnel de santé."},
     { "question": "Est-ce que les programmes de formation payants sont conformes aux lois en vigueur ?", "reponse": "Oui, tant que le programme est de qualité et apporte une réelle valeur ajoutée au professionnel de santé. Les frais doivent être justifiés, et le programme doit respecter les règles de transparence et de justification des dépenses."},
     { "question": "Est-ce acceptable d’offrir un repas lors d'une rencontre professionnelle ?", "reponse": "Un repas peut être offert dans le cadre d’une réunion professionnelle, mais il doit être modéré et justifié par l'objectif de la rencontre. Le coût du repas doit être raisonnable et ne doit pas créer de pression ou d’influence sur le professionnel."},
     { "question": "Puis-je rémunérer un médecin pour un article ou une publication scientifique ?", "reponse": "Oui, la rémunération est possible si elle est en rapport avec un travail scientifique concret (rédaction d'articles, recherches, etc.). La rémunération doit être proportionnée à la tâche effectuée, et la relation doit être transparente et bien documentée."},
     { "question": "Est-il autorisé de proposer un financement pour une étude clinique menée par un médecin ?", "reponse": "Oui, mais cela doit être fait dans un cadre strictement scientifique et éthique. Les financements doivent être justifiés et ne doivent pas être utilisés pour influencer le médecin dans ses pratiques professionnelles. La conformité avec les normes de bonne pratique clinique et les règles de transparence est essentielle."},
     { "question": "Les séminaires internes peuvent-ils inclure des repas ou des divertissements pour les médecins ?", "reponse": " Oui, à condition que les repas ou divertissements soient raisonnables et directement liés à l’objectif professionnel de la rencontre. Il est important que ces dépenses ne soient pas excessives et soient justifiées par la nature de l’événement."},
     { "question": "Puis-je inviter un médecin à participer à une étude de marché rémunérée ?", "reponse": "Oui, à condition que l’étude soit sérieuse, professionnelle et qu’elle ne constitue pas une tentative d’influencer les décisions médicales. La rémunération doit être en adéquation avec le temps consacré à l’étude et être documentée."},
     { "question": "Est-il acceptable d’offrir un voyage pour un congrès à un médecin ?", "reponse": "Oui, un voyage peut être pris en charge, mais seulement s’il est justifié par l’intérêt scientifique du congrès. Les frais doivent être raisonnables, et le voyage ne doit pas dépasser les limites de ce qui est nécessaire pour la participation à l’événement."},
     { "question": "Puis-je organiser une réunion d'affaires avec des médecins à l’extérieur de leur lieu de travail ?", "reponse": "Oui, tant que la réunion est axée sur des objectifs professionnels et que les frais sont raisonnables. Il est important que les circonstances de la réunion ne soient pas perçues comme une tentative d’obtenir des avantages personnels pour les médecins."},
     { "question": "Les pratiques de parrainage d’événements scientifiques sont-elles autorisées ?", "reponse": "Le parrainage d’événements scientifiques est autorisé, mais il doit être transparent, justifié par l’aspect scientifique de l’événement, et ne doit pas avoir pour but d'influencer la pratique médicale. Les financements doivent être publiés conformément aux exigences légales."},
     { "question": "Puis-je offrir des billets pour des événements sportifs à un médecin ?", "reponse": "Non, les billets pour des événements non liés à un objectif professionnel (comme des événements sportifs) ne sont généralement pas considérés comme conformes à la loi. Cela peut être perçu comme une tentative d’influencer un médecin, ce qui peut poser un problème de conformité."},
     { "question": "Est-il autorisé de sponsoriser un médecin pour une conférence internationale ?", "reponse": "Oui, un laboratoire peut sponsoriser un médecin pour une conférence internationale si la conférence est liée à son domaine d'expertise et si le financement est limité aux frais professionnels. La transparence sur les montants et les avantages accordés est primordiale."},
     { "question": "Est-ce que je peux organiser des groupes de discussion avec des médecins dans un cadre informel ?", "reponse": "Oui, mais l’objectif doit rester professionnel. Les discussions doivent porter sur des sujets scientifiques ou médicaux pertinents, et les frais doivent être modérés et clairement documentés."},
     { "question": "Puis-je organiser une visite de laboratoire pour des médecins ?", "reponse": "Oui, les visites de laboratoire peuvent être organisées, mais elles doivent avoir un but éducatif ou scientifique. Les frais doivent être raisonnables et la visite doit respecter les principes de transparence."},
     { "question": "Est-il possible de rémunérer un médecin pour sa participation à un panel de discussion ?", "reponse": "Oui, un médecin peut être rémunéré pour sa participation à un panel de discussion, mais la rémunération doit être proportionnée au temps et à l'expertise fournis. Les conditions de la rémunération doivent être claires, et tout paiement doit être documenté."},

    # Ajoutez d'autres questions-réponses ici
]

# Créer un DataFrame
faq_df = pd.DataFrame(data)
faq_df.head()

,question,reponse
0,Est-ce que je peux inviter un médecin à dîner ?,"Oui, mais seulement sous certaines conditions...."
1,Puis-je offrir un cadeau à un médecin ou à un ...,"Les cadeaux doivent être modestes, liés à une ..."
2,Est-il acceptable de financer la participation...,"Oui, cela est possible, mais cela doit être cl..."
3,Peut-on offrir une rémunération à un médecin p...,"Oui, un paiement pour une expertise scientifiq..."
4,Est-il permis de prendre en charge les frais d...,"Oui, cela est possible si les frais sont direc..."


In [8]:
df = faq_df

## Preparation de text

* Nettoyage (ponctuation, espaces)
* Conversion en minuscules
* Suppression des mots vides (optionnel)
* Lemmatisation (optionnel)

In [9]:
# Fonction de prétraitement
def preprocess_text(sentence):
    # Tokenisation
    # print(sentence)
    #sentence = sentence.replace('-', ' ').replace('+', ' ').replace('/', ' ').replace('#', ' ')
    #sentence = word_tokenize(sentence)
    
    # Convertir en minuscules
    sentence = sentence.lower()
    
    # Enlever la ponctuation et les caractères spéciaux
    sentence = re.sub(r'[^\w\s]', ' ', sentence)
    
    # Enlever les espaces multiples
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    
    # Supprimer les stopwords
    stop_words = set(stopwords.words('french'))
    words = sentence.split()
    words = [word for word in words if word not in stop_words]

    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    sentence = ' '.join(words)
    return sentence
    

In [10]:
# Appliquer à chaque question dans la FAQ
df['question_normalized'] = df['question'].apply(preprocess_text)
df.head()

,question,reponse,question_normalized
0,Est-ce que je peux inviter un médecin à dîner ?,"Oui, mais seulement sous certaines conditions....",peux inviter médecin dîner
1,Puis-je offrir un cadeau à un médecin ou à un ...,"Les cadeaux doivent être modestes, liés à une ...",puis offrir cadeau médecin professionnel santé
2,Est-il acceptable de financer la participation...,"Oui, cela est possible, mais cela doit être cl...",acceptable financer participation médecin congrès
3,Peut-on offrir une rémunération à un médecin p...,"Oui, un paiement pour une expertise scientifiq...",peut offrir rémunération médecin consultation ...
4,Est-il permis de prendre en charge les frais d...,"Oui, cela est possible si les frais sont direc...",permis prendre charge frais transport médecin ...


## SentenceTransformers
Les modèles de similarité basés sur des embeddings, comme ceux de sentence-transformers, sont conçus pour saisir le sens global d'une phrase. Ainsi, même avec une formulation légèrement différente, ils devraient pouvoir reconnaître les questions similaires. Cependant, utiliser des modèles plus puissants, comme paraphrase-multilingual-MiniLM-L12-v2 (de la bibliothèque sentence-transformers), peut améliorer les résultats.

In [11]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle spécialisé dans la paraphrase
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

### Extension avec la Recherche de Similarité Floue
Une autre technique consiste à utiliser des méthodes de similarité floue (fuzzy matching) pour compléter la similarité sémantique. La bibliothèque fuzzywuzzy peut aider à capturer des variations textuelles mineures.

In [12]:
# Fonction pour vérifier la similarité floue en complément
def fuzzy_similarity(question1, question2):
    return fuzz.ratio(preprocess_text(question1), preprocess_text(question2)) / 100

In [13]:
def find_similar_question(user_question, faq_df, model, threshold=0.95):
    # Prétraiter la question de l'utilisateur
    user_question_normalized = preprocess_text(user_question)
    
    # Encoder la question normalisée
    user_embedding = model.encode(user_question_normalized)

    # Calculer la similarité entre la question de l'utilisateur et chaque question de la FAQ
    faq_df['similarity'] = faq_df['question_normalized'].apply(
        lambda x: util.pytorch_cos_sim(user_embedding, model.encode(x)).item()
    )
    
    # Vérifier également la similarité floue
    faq_df['fuzzy_similarity'] = faq_df['question_normalized'].apply(
        lambda x: fuzzy_similarity(user_question_normalized, x)
    )

    # Combiner les deux scores (pondérés si nécessaire)
    faq_df['combined_similarity'] = (faq_df['similarity'] + faq_df['fuzzy_similarity']) / 2

    # Trouver la question avec la similarité combinée la plus élevée
    best_match = faq_df.loc[faq_df['combined_similarity'].idxmax()]

    # Si la similarité combinée dépasse le seuil, retourner la réponse correspondante
    if best_match['combined_similarity'] >= threshold:
        return best_match['reponse']
    else:
        return "Je n'ai pas de réponse exacte pour cette question, pouvez-vous reformuler ?"

In [14]:
# Exemple de question posée par l'utilisateur
user_question = "Est-ce que je peux inviter un médecin à dîner ?"

# Obtenir la réponse
response = find_similar_question(user_question, df, model)
print("Réponse :", response)

Réponse : Oui, mais seulement sous certaines conditions. L'invitation doit être raisonnable, proportionnée et liée à un objectif professionnel clair, comme une présentation scientifique ou une discussion sur un produit. Elle doit également respecter les règles de transparence (enregistrement et déclaration des avantages accordés). De plus, elle ne doit pas être perçue comme une tentative d’influencer les décisions médicales.


In [15]:
user_question = "Puis-je offrir un cadeau à un médecin ou à un professionnel de santé ?"

# Obtenir la réponse
response = find_similar_question(user_question, faq_df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [16]:
user_question = "puis-je offrir un cadeau à un médecin ou à un professionnel de santé"

# Obtenir la réponse
response = find_similar_question(user_question, faq_df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [17]:
user_question = "Puis je offrir un cadeau à un médecin ou à un professionnel de santé ?"

# Obtenir la réponse
response = find_similar_question(user_question, faq_df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [18]:
user_question = "puis offrir cadeau médecin professionnel santé?"
# Obtenir la réponse
response = find_similar_question(user_question, faq_df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [19]:
user_question = "puis-je offrir un cadeau à un médecin ou à un professionnel de santé"

# Obtenir la réponse
response = find_similar_question(user_question, faq_df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [53]:
# Liste de questions-réponses
data = [
    { 
    "question": "Est-ce que je peux inviter un médecin à dîner ?", "reponse": "Oui, mais seulement sous certaines conditions. L'invitation doit être raisonnable, proportionnée et liée à un objectif professionnel clair, comme une présentation scientifique ou une discussion sur un produit. Elle doit également respecter les règles de transparence (enregistrement et déclaration des avantages accordés). De plus, elle ne doit pas être perçue comme une tentative d’influencer les décisions médicales.",
     "aliases":["Est-ce que je peux inviter un doctor à dîner ?", "Est-ce que je peux inviter un médecin à manger ?"]
    },
    { "question": "Puis-je offrir un cadeau à un médecin ou à un professionnel de santé ?", "reponse": "Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.",
    "aliases":["Puis-je offrir un cadeau à un doctor ou à un professionnel de santé ?", "Puis-je offrir un petit cadeau à un doctor ou à un professionnel de santé ?"]},
    { "question": "Est-il acceptable de financer la participation d’un médecin à un congrès ?", "reponse": "Oui, cela est possible, mais cela doit être clairement lié à une activité scientifique ou professionnelle. Les frais doivent être justifiés, la participation doit être documentée, et le médecin ne doit pas être surchargé par des financements en dehors de ce cadre. Toute contribution doit être publiée selon les exigences de la loi Sapin 2.",
     "aliases": "non" }
    # Ajoutez d'autres questions-réponses ici
]

# Créer un DataFrame
faq_df = pd.DataFrame(data)
faq_df.head()

,question,reponse,aliases
0,Est-ce que je peux inviter un médecin à dîner ?,"Oui, mais seulement sous certaines conditions....",[Est-ce que je peux inviter un doctor à dîner ...
1,Puis-je offrir un cadeau à un médecin ou à un ...,"Les cadeaux doivent être modestes, liés à une ...",[Puis-je offrir un cadeau à un doctor ou à un ...
2,Est-il acceptable de financer la participation...,"Oui, cela est possible, mais cela doit être cl...",non


In [54]:
df = faq_df
df = df.reset_index()
df=df.rename(columns={"index":"id"})
df.head()

,id,question,reponse,aliases
0,0,Est-ce que je peux inviter un médecin à dîner ?,"Oui, mais seulement sous certaines conditions....",[Est-ce que je peux inviter un doctor à dîner ...
1,1,Puis-je offrir un cadeau à un médecin ou à un ...,"Les cadeaux doivent être modestes, liés à une ...",[Puis-je offrir un cadeau à un doctor ou à un ...
2,2,Est-il acceptable de financer la participation...,"Oui, cela est possible, mais cela doit être cl...",non


In [55]:
# Appliquer à chaque question dans la FAQ
df['question_normalized'] = df['question'].apply(preprocess_text)
df['aliases_normalized'] = df['aliases'].apply(lambda aliases: [preprocess_text(alias) for alias in aliases])
df.head()

,id,question,reponse,aliases,question_normalized,aliases_normalized
0,0,Est-ce que je peux inviter un médecin à dîner ?,"Oui, mais seulement sous certaines conditions....",[Est-ce que je peux inviter un doctor à dîner ...,peux inviter médecin dîner,"[peux inviter doctor dîner, peux inviter médec..."
1,1,Puis-je offrir un cadeau à un médecin ou à un ...,"Les cadeaux doivent être modestes, liés à une ...",[Puis-je offrir un cadeau à un doctor ou à un ...,puis offrir cadeau médecin professionnel santé,[puis offrir cadeau doctor professionnel santé...
2,2,Est-il acceptable de financer la participation...,"Oui, cela est possible, mais cela doit être cl...",non,acceptable financer participation médecin congrès,"[, o, ]"


In [56]:
from sentence_transformers import SentenceTransformer, util

# Charger un modèle spécialisé dans la paraphrase
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [57]:
# Fonction pour vérifier la similarité floue en complément
def fuzzy_similarity(question1, question2):
    return fuzz.ratio(preprocess_text(question1), preprocess_text(question2)) / 100

In [58]:
def find_similar_question(user_question, faq_df, model, threshold=0.95):
    # Prétraiter la question de l'utilisateur
    user_question_normalized = preprocess_text(user_question)
    
    # Encoder la question normalisée
    user_embedding = model.encode(user_question_normalized)
    
    # Calculer la similarité entre la question de l'utilisateur et chaque question ou alias de la FAQ
    faq_df['similarity'] = faq_df.apply(
        lambda row: max(
            util.pytorch_cos_sim(user_embedding, model.encode(row['question_normalized'])).item(),
            *[
                util.pytorch_cos_sim(user_embedding, model.encode(alias)).item()
                for alias in row['aliases_normalized']
            ]
        ),
        axis=1
    )
    
    # Vérifier également la similarité floue pour chaque question et alias
    faq_df['fuzzy_similarity'] = faq_df.apply(
        lambda row: max(
            fuzzy_similarity(user_question_normalized, row['question_normalized']),
            *[fuzzy_similarity(user_question_normalized, alias) for alias in row['aliases_normalized']]
        ),
        axis=1
    )

    # Combiner les deux scores de similarité (pondérés si nécessaire)
    faq_df['combined_similarity'] = (faq_df['similarity'] + faq_df['fuzzy_similarity']) / 2

    # Trouver la question avec la similarité combinée la plus élevée
    best_match = faq_df.loc[faq_df['combined_similarity'].idxmax()]

    # Si la similarité combinée dépasse le seuil, retourner la réponse correspondante
    if best_match['combined_similarity'] >= threshold:
        return best_match['reponse']
    else:
        return "Je n'ai pas de réponse exacte pour cette question, pouvez-vous reformuler ?"

In [59]:
# Exemple de question posée par l'utilisateur
user_question = "Est-ce que je peux inviter un médecin à dîner ?"

# Obtenir la réponse
response = find_similar_question(user_question, df, model)
print("Réponse :", response)

Réponse : Oui, mais seulement sous certaines conditions. L'invitation doit être raisonnable, proportionnée et liée à un objectif professionnel clair, comme une présentation scientifique ou une discussion sur un produit. Elle doit également respecter les règles de transparence (enregistrement et déclaration des avantages accordés). De plus, elle ne doit pas être perçue comme une tentative d’influencer les décisions médicales.


In [61]:
# Exemple de question posée par l'utilisateur
user_question = "Est-ce que je peux inviter un doctor à dîner ?"

# Obtenir la réponse
response = find_similar_question(user_question, df, model)
print("Réponse :", response)

Réponse : Oui, mais seulement sous certaines conditions. L'invitation doit être raisonnable, proportionnée et liée à un objectif professionnel clair, comme une présentation scientifique ou une discussion sur un produit. Elle doit également respecter les règles de transparence (enregistrement et déclaration des avantages accordés). De plus, elle ne doit pas être perçue comme une tentative d’influencer les décisions médicales.


In [62]:
# Exemple de question posée par l'utilisateur
user_question = "Puis-je offrir un cadeau à un doctor ou à un professionnel de santé ?"

# Obtenir la réponse
response = find_similar_question(user_question, df, model)
print("Réponse :", response)

Réponse : Les cadeaux doivent être modestes, liés à une activité professionnelle, et ne doivent pas avoir pour but d'influencer la prescription de produits. Il est important de documenter ces cadeaux et de respecter les règles internes de l'entreprise et les obligations légales, telles que la loi Sapin 2, qui exige la transparence sur ces pratiques.


In [23]:
aliases = {
    "Est-ce que je peux inviter un médecin à dîner ?":["Est-ce que je peux inviter un doctor à dîner ?",
                                                      "Est-ce que je peux inviter un médecin à manger ?"],
    "Puis-je offrir un cadeau à un médecin ou à un professionnel de santé ?": ["Puis-je offrir un cadeau à un doctor ou à un professionnel de santé ?",
                                                             "Puis-je offrir un petit cadeau à un doctor ou à un professionnel de santé ?"
                                                                               
    ]
}

In [24]:
# Prétraitement : Créer des embeddings pour chaque question maîtresse et ses alias
def create_embedding_dict(faq, aliases):
    embedding_dict = {}
    for question, response in faq.items():
        # Encoder la question principale
        embedding_dict[question] = model.encode(question)
        # Encoder chaque alias de cette question
        for alias in aliases.get(question, []):
            embedding_dict[alias] = model.encode(alias)
    return embedding_dict

embedding_dict = create_embedding_dict(df, aliases)

In [27]:
embedding_dict

{'question': array([ 5.51723897e-01,  1.41026258e-01,  1.37360981e-02, -1.09438598e-01,
        -1.30279809e-01, -3.88246626e-01,  3.12992573e-01,  2.57388532e-01,
        -8.33623782e-02, -7.34273121e-02, -6.50971383e-03, -2.11568430e-01,
        -2.95022782e-02, -1.25246152e-01, -5.44672608e-02,  8.92884508e-02,
        -7.42767677e-02, -3.12628329e-01, -4.25855964e-01,  2.14469612e-01,
         3.98768902e-01, -1.71661973e-01,  1.59644950e-02, -1.69625077e-02,
        -2.21486315e-01, -3.64231206e-02, -6.85035139e-02,  9.06536356e-02,
        -8.04348812e-02, -2.58303344e-01, -2.04941630e-02, -1.21573769e-01,
         7.66097605e-02,  1.68920398e-01,  2.05374256e-01,  2.31734470e-01,
         5.90134077e-02, -3.94772105e-02,  9.94445905e-02, -4.09333669e-02,
         4.55269627e-02, -2.44162872e-01,  1.04303785e-01,  9.14386138e-02,
         1.38397500e-01,  2.19028756e-01,  3.91588025e-02,  1.29366413e-01,
        -1.17754668e-01, -1.05109485e-02,  2.94149965e-02, -1.83827147e-01,


In [37]:
# Prétraitement : Créer des embeddings pour chaque question maîtresse et ses alias
def create_embedding_dict(faq, aliases):
    embedding_dict = {}
    for question, response in faq.items():
        # Encoder la question principale
        embedding_dict[question] = model.encode(question)
        # Encoder chaque alias de cette question
        for alias in aliases.get(question, []):
            embedding_dict[alias] = model.encode(alias)
    return embedding_dict

embedding_dict = create_embedding_dict(df, aliases)

# Fonction pour trouver la meilleure correspondance
def find_best_match(user_question, embedding_dict, threshold=0.7):
    user_embedding = model.encode(user_question)
    
    best_match = None
    best_similarity = 0

    # Calculer la similarité avec chaque question ou alias
    for question, embedding in embedding_dict.items():
        similarity = util.pytorch_cos_sim(user_embedding, embedding).item()
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = question

    # Vérifier si la similarité dépasse le seuil
    if best_similarity >= threshold:
        return best_match
    else:
        return None

# Fonction principale pour obtenir la réponse
def get_response(user_question, faq, embedding_dict, threshold=0.7):
    # Trouver la question maîtresse ou l'alias correspondant
    best_match = find_best_match(user_question, embedding_dict, threshold)
    if best_match:
        # Si un alias correspond, retourner la réponse de la question maîtresse
        main_question = best_match if best_match in faq else None
        # Associer l'alias trouvé avec sa question maîtresse
        if main_question is None:
            main_question = next((q for q, a in aliases.items() if best_match in a), None)
        return faq.get(main_question, "Désolé, je n'ai pas de réponse pour cette question.")
    else:
        return "Désolé, je n'ai pas de réponse pour cette question. Pouvez-vous reformuler ?"

# Exemple d'utilisation
user_question = "Est-ce que je peux inviter un médecin à dîner ?"
response = get_response(user_question, df, embedding_dict)
print("Réponse :", response)

Réponse : Désolé, je n'ai pas de réponse pour cette question. Pouvez-vous reformuler ?


In [31]:
# Fonction pour trouver la meilleure correspondance
def find_best_match(user_question, embedding_dict, threshold=0.7):
    user_embedding = model.encode(user_question)
    
    best_match = None
    best_similarity = 0

    # Calculer la similarité avec chaque question ou alias
    for question, embedding in embedding_dict.items():
        similarity = util.pytorch_cos_sim(user_embedding, embedding).item()
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = question

    # Vérifier si la similarité dépasse le seuil
    if best_similarity >= threshold:
        return best_match
    else:
        return None


In [32]:
def find_similar_question(user_question, faq_df, embedding_dict, model, threshold=0.95):
    # Prétraiter la question de l'utilisateur
    user_question_normalized = preprocess_text(user_question)
    
    # Encoder la question normalisée
    user_embedding = model.encode(user_question_normalized)

    # Calculer la similarité entre la question de l'utilisateur et chaque question de la FAQ
    faq_df['similarity'] = faq_df['question_normalized'].apply(
        lambda x: util.pytorch_cos_sim(user_embedding, model.encode(x)).item()
    )
    
    # Vérifier également la similarité floue
    faq_df['fuzzy_similarity'] = faq_df['question_normalized'].apply(
        lambda x: fuzzy_similarity(user_question_normalized, x)
    )

    # Combiner les deux scores (pondérés si nécessaire)
    faq_df['combined_similarity'] = (faq_df['similarity'] + faq_df['fuzzy_similarity']) / 2

    # Trouver la question avec la similarité combinée la plus élevée
    best_match = faq_df.loc[faq_df['combined_similarity'].idxmax()]

    # Si la similarité combinée dépasse le seuil, retourner la réponse correspondante
    if best_match['combined_similarity'] >= threshold:
        return best_match['reponse']
    else:
        return "Je n'ai pas de réponse exacte pour cette question, pouvez-vous reformuler ?"

    # Trouver la question maîtresse ou l'alias correspondant
    best_match = find_best_match(user_question, embedding_dict, threshold)
    if best_match:
        # Si un alias correspond, retourner la réponse de la question maîtresse
        main_question = best_match if best_match in faq else None
        # Associer l'alias trouvé avec sa question maîtresse
        if main_question is None:
            main_question = next((q for q, a in aliases.items() if best_match in a), None)
        return faq.get(main_question, "Désolé, je n'ai pas de réponse pour cette question.")
    else:
        return "Désolé, je n'ai pas de réponse pour cette question. Pouvez-vous reformuler ?"

In [ ]:
# Fonction principale pour obtenir la réponse
def get_response(user_question, faq, embedding_dict, threshold=0.7):
    # Trouver la question maîtresse ou l'alias correspondant
    best_match = find_best_match(user_question, embedding_dict, threshold)
    if best_match:
        # Si un alias correspond, retourner la réponse de la question maîtresse
        main_question = best_match if best_match in faq else None
        # Associer l'alias trouvé avec sa question maîtresse
        if main_question is None:
            main_question = next((q for q, a in aliases.items() if best_match in a), None)
        return faq.get(main_question, "Désolé, je n'ai pas de réponse pour cette question.")
    else:
        return "Désolé, je n'ai pas de réponse pour cette question. Pouvez-vous reformuler ?"

In [33]:
# Exemple d'utilisation
user_question = "Est-ce que je peux inviter un médecin à dîner ?"
response = get_response(user_question, df, embedding_dict, model)
print("Réponse :", response)

TypeError: '>=' not supported between instances of 'float' and 'SentenceTransformer'

In [ ]:
# Point de terminaison pour poser une question
@app.post("/faq/")
async def get_faq_answer(request: FAQRequest):
    try:
        # Utiliser la fonction de recherche de la question similaire
        answer = find_similar_question(request.question, faq_df, model)
        return {"question": request.question, "answer": answer}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
    